In [1]:
!pip install -q transformers datasets evaluate bert_score detoxify

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset, DatasetDict
import numpy as np
from evaluate import load

# 1. Load and prepare dataset
dataset = load_dataset("Rhma/DIALOCONAN")
small_dataset = dataset["train"].select(range(3500))

# Split train data into train/validation
train_val = small_dataset.train_test_split(test_size=0.15, seed=42)
dataset = DatasetDict({
    "train": train_val["train"],
    "validation": train_val["test"]
})

# 2. Group turns by dialogue_id
def group_dialogues(examples):
    sorted_data = sorted(zip(examples["dialogue_id"], 
                            examples["turn_id"], 
                            examples["text"],
                            examples["type"],
                            examples["TARGET"]),
                       key=lambda x: (x[0], x[1]))
    dialogues = []
    current_dialogue = []
    current_id = None
    for item in sorted_data:
        dialogue_id, turn_id, text, turn_type, target = item
        if dialogue_id != current_id:
            if current_id is not None and current_dialogue:
                dialogues.append({
                    "dialogue_id": current_id,
                    "turns": current_dialogue,
                    "target": current_dialogue[0]["target"]
                })
            current_id = dialogue_id
            current_dialogue = []
        current_dialogue.append({
            "text": text,
            "type": turn_type,
            "target": target
        })
    if current_id is not None and current_dialogue:
        dialogues.append({
            "dialogue_id": current_id,
            "turns": current_dialogue,
            "target": current_dialogue[0]["target"]
        })
    return {"dialogues": dialogues}

processed_dataset = dataset.map(
    group_dialogues,
    batched=True,
    remove_columns=dataset["train"].column_names,
    batch_size=1000
)

# 3. Create conversation history for each CN turn
def create_conversation_history(examples):
    new_examples = {"input": [], "target": []}
    for dialogue in examples["dialogues"]:
        history = []
        for turn in dialogue["turns"]:
            if turn["type"] == "CN":
                new_examples["input"].append(" [SEP] ".join(history))
                new_examples["target"].append(turn["text"])
            history.append(turn["text"])
    return new_examples

final_dataset = processed_dataset.map(
    create_conversation_history,
    batched=True,
    remove_columns=["dialogues"]
)

# 4. Load pretrained BLOOMZ model and tokenizer
model_name = "bigscience/bloomz-3b"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16)

# 5. Generation function (inference only)
def generate_counterspeech(dialogue_history):
    device = model.device
    input_text = " [SEP] ".join(dialogue_history) + " [ANS] "
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)
    with torch.inference_mode():
        outputs = model.generate(
            inputs.input_ids,
            max_new_tokens=128,
            num_beams=5,
            repetition_penalty=2.0,
            early_stopping=True
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# 6. Example usage
sample_dialogue = [
    "You people are ruining our country!",
    "Immigrants are stealing our jobs!",
    "We should send them all back!"
]
print("\nGenerated counterspeech:")
print(generate_counterspeech(sample_dialogue))


2025-05-08 08:06:22.847056: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746691582.869404     136 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746691582.876300     136 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


README.md:   0%|          | 0.00/452 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16625 [00:00<?, ? examples/s]

Map:   0%|          | 0/2975 [00:00<?, ? examples/s]

...

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.01G [00:00<?, ?B/s]


Generated counterspeech:
You people are ruining our country! [SEP] Immigrants are stealing our jobs! [SEP] We should send them all back! [ANS]  No, we will not send them back.


In [ ]:
# Install dependencies if not already installed
!pip install -q evaluate detoxify tqdm

from evaluate import load
from detoxify import Detoxify
from tqdm import tqdm
import numpy as np

# Load metrics
bertscore = load("bertscore")
meteor = load("meteor")
bleu = load("bleu")

# Use first 100 samples
inputs = [ex["input"] for ex in final_dataset["validation"]][:100]
targets = [ex["target"] for ex in final_dataset["validation"]][:100]

# Generate predictions
print("Generating counter speech...")
generated = []
for text in tqdm(inputs, desc="Generating"):
    response = generate_counterspeech(text)  # <-- make sure this function is defined
    generated.append(response)

# BERTScore
print("Calculating BERTScore...")
bertscore_result = bertscore.compute(
    predictions=generated,
    references=targets,
    model_type="distilbert-base-uncased"
)
print(f"BERTScore F1: {np.mean(bertscore_result['f1']):.4f}")

# METEOR
print("Calculating METEOR...")
meteor_result = meteor.compute(predictions=generated, references=targets)
print(f"METEOR: {meteor_result['meteor']:.4f}")

# BLEU
print("Calculating BLEU...")
bleu_result = bleu.compute(predictions=generated, references=[[ref] for ref in targets])
print(f"BLEU: {bleu_result['bleu']:.4f}")

# Toxicity
print("Calculating Toxicity...")
toxicity_scores = [Detoxify('original').predict(pred)['toxicity'] for pred in tqdm(generated, desc="Toxicity")]
avg_toxicity = np.mean(toxicity_scores)
print(f"Avg. Toxicity Score: {avg_toxicity:.4f}")


In [3]:
# Install dependencies if not already installed
!pip install -q evaluate detoxify tqdm
!pip install rouge_score bert_score
from evaluate import load
from detoxify import Detoxify
from tqdm import tqdm
import numpy as np
import math

# Load metrics
rouge = load("rouge")
bertscore = load("bertscore")
# Use first 100 samples
inputs = [ex["input"] for ex in final_dataset["validation"]][:100]
targets = [ex["target"] for ex in final_dataset["validation"]][:100]

# Generate predictions
print("Generating counter speech...")
generated = []
for text in tqdm(inputs, desc="Generating"):
    response = generate_counterspeech(text)  # <-- make sure this function is defined
    generated.append(response)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Generating counter speech...


Generating: 100%|██████████| 100/100 [06:51<00:00,  4.12s/it]

Calculating BERTScore...


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

BERTScore F1: 0.5972
Calculating ROUGE...


AttributeError: 'numpy.float64' object has no attribute 'fmeasure'

In [4]:
# BERTScore
print("Calculating BERTScore...")
bertscore_result = bertscore.compute(
    predictions=generated,
    references=targets,
    model_type="distilbert-base-uncased"
)
print(f"BERTScore F1: {np.mean(bertscore_result['f1']):.4f}")

# ROUGE
# ROUGE
print("Calculating ROUGE...")
rouge_result = rouge.compute(predictions=generated, references=targets)
print(f"ROUGE-1 F1: {rouge_result['rouge1']:.4f}")
print(f"ROUGE-2 F1: {rouge_result['rouge2']:.4f}")
print(f"ROUGE-L F1: {rouge_result['rougeL']:.4f}")
# Perplexity
print("Calculating Perplexity...")
def calculate_perplexity(texts):
    total_log_prob = 0.0
    total_words = 0
    for text in texts:
        words = text.split()
        total_words += len(words)
        # You can use a pre-trained language model (e.g., GPT-2) for calculating perplexity
        # Here, we will use a placeholder for the log-prob calculation, which should ideally come from a language model
        # For simplicity, assume a fixed value here
        total_log_prob += len(words) * math.log(1.0)  # Placeholder for log-prob calculation
    return math.exp(-total_log_prob / total_words) if total_words > 0 else float('inf')

perplexity_result = calculate_perplexity(generated)
print(f"Perplexity: {perplexity_result:.4f}")

# Toxicity
print("Calculating Toxicity...")
toxicity_scores = [Detoxify('original').predict(pred)['toxicity'] for pred in tqdm(generated, desc="Toxicity")]
avg_toxicity = np.mean(toxicity_scores)
print(f"Avg. Toxicity Score: {avg_toxicity:.4f}")

Calculating BERTScore...
BERTScore F1: 0.5972
Calculating ROUGE...
ROUGE-1 F1: 0.0332
ROUGE-2 F1: 0.0005
ROUGE-L F1: 0.0307
Calculating Perplexity...
Perplexity: 1.0000
Calculating Toxicity...


Toxicity:   0%|          | 0/100 [00:00<?, ?it/s]Downloading: "https://github.com/unitaryai/detoxify/releases/download/v0.1-alpha/toxic_original-c1212f89.ckpt" to /root/.cache/torch/hub/checkpoints/toxic_original-c1212f89.ckpt

  0%|          | 0.00/418M [00:00<?, ?B/s]
  3%|▎         | 14.4M/418M [00:00<00:02, 150MB/s]
 12%|█▏        | 50.1M/418M [00:00<00:01, 278MB/s]
 21%|██        | 85.8M/418M [00:00<00:01, 321MB/s]
 28%|██▊       | 116M/418M [00:00<00:01, 287MB/s] 
 35%|███▍      | 144M/418M [00:00<00:01, 256MB/s]
 41%|████▏     | 173M/418M [00:00<00:00, 267MB/s]
 50%|████▉     | 208M/418M [00:00<00:00, 297MB/s]
 57%|█████▋    | 238M/418M [00:00<00:00, 288MB/s]
 65%|██████▌   | 274M/418M [00:00<00:00, 313MB/s]
 73%|███████▎  | 304M/418M [00:01<00:00, 302MB/s]
 82%|████████▏ | 341M/418M [00:01<00:00, 327MB/s]
 89%|████████▉ | 373M/418M [00:01<00:00, 323MB/s]
100%|██████████| 418M/418M [00:01<00:00, 278MB/s]


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Toxicity: 100%|██████████| 100/100 [00:45<00:00,  2.19it/s]

Avg. Toxicity Score: 0.0020
